## Prompt
A LangChain prompt template is a reusable way to build prompts for language models. It gives you a clear structure to follow, while making it easy to customize prompts and save time.

LangChain’s templates help keep your prompts consistent, and make the model’s answers more predictable and reliable. Over time, they also improve results by reducing randomness in how prompts are written.

A prompt template usually consists of two things:

1. A text prompt, which is just a chunk of natural language. It can be plain text, or it can have placeholders like {variable} that get filled in with real values when you use it.

2. Optional formatting rules, so you can control how the final prompt looks, like whether text should be bold, in all caps, or styled a certain way.

## 3 Types of LangChain Prompt Templates
LangChain offers different template classes:

- String PromptTemplate for creating basic prompts.
- ChatPromptTemplate for chat-based prompts with multiple messages.
- MessagesPlaceholder for injecting a dynamic list of messages (such as a conversation history).

### PromptTemplate: Simple String-Based Prompts
These prompt templates are used to format a single string, and generally are used for simpler inputs. 

In [2]:
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template("Share an interesting fact about {animal}.")  # infers 'animal' as input variable

# Format the template with a specific animal
filled_prompt = example_prompt.format(animal="octopus")
print(filled_prompt)



Share an interesting fact about octopus.


### ChatPromptTemplate: Chat-Style Prompt with Roles
These prompt templates are used to format a list of messages. These "templates" consist of a list of templates themselves.

In [3]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a patient tutor who explains things clearly."),
    ("human", "Can you explain {concept} like I'm five?")
])

# Fill in the template with a specific concept
formatted_messages = chat_prompt.format_messages(concept="gravity")

print(formatted_messages)

[SystemMessage(content='You are a patient tutor who explains things clearly.', additional_kwargs={}, response_metadata={}), HumanMessage(content="Can you explain gravity like I'm five?", additional_kwargs={}, response_metadata={})]


In [2]:
## Chat Completion Model
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chatmodel = ChatGroq(model="qwen/qwen3-32b")

Chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "you are an {profession} expert on {topic}"),
        ("human", "Hello, Mr.{profession} can you please answer a question ?"),
        ('ai', "Sure, I'd be happy to help!"),
        ("human", "{user_input}"),
    ]
)

message = Chat_template.format(
    profession="doctor",
    topic="healthcare",
    user_input="What are the symptoms of flu?"
)

llm = ChatGroq(model="qwen/qwen3-32b")
llm_model_prompt = message

print(llm.invoke(llm_model_prompt).content)


<think>
Okay, the user is asking about the symptoms of the flu. Let me start by recalling the common flu symptoms. The main ones include fever, cough, sore throat, runny or stuffy nose, muscle or body aches, headaches, fatigue, and sometimes vomiting or diarrhea, especially in kids. I should probably list these out clearly.

Wait, the user might be differentiating between a cold and the flu. I should mention that the flu tends to come on more suddenly and is more severe than a cold. Also, fever is a key differentiator. Oh, and it's important to note that not everyone with the flu will have a fever, but it's common.

I should also think about when to see a doctor. Severe symptoms like difficulty breathing, chest pain, or persistent vomiting are red flags. Maybe include that advice as well.

Let me check if there's anything else. The flu can lead to complications like pneumonia, so highlighting the importance of rest and hydration is good. Also, maybe mention the duration—symptoms usuall

### MessagesPlaceholder: Inserting Dynamic Chat History into a Prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful career coach."),
    MessagesPlaceholder("conversation"),  # Dynamic history insertion
    ("human", "{current_question}")
])

# Define history using proper message objects
conversation_history = [
    HumanMessage(content="How do I prepare for a job interview?"),
    AIMessage(content="Start by researching the company and practicing common questions.")
]

formatted_messages = chat_prompt.format_messages(
    conversation=conversation_history,
    current_question="Should I send a thank-you email afterward?"
)

print(formatted_messages)

[SystemMessage(content='You are a helpful career coach.', additional_kwargs={}, response_metadata={}), HumanMessage(content='How do I prepare for a job interview?', additional_kwargs={}, response_metadata={}), AIMessage(content='Start by researching the company and practicing common questions.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Should I send a thank-you email afterward?', additional_kwargs={}, response_metadata={})]


: 

### Few Shot Prompting 
providing clear and effective instructions is paramount to achieving desired outcomes. "Few-shot prompting" is a powerful technique that guides an LLM by giving it a small number of examples of the task you want it to perform. This approach helps the model understand the context and desired output format, often leading to significantly better and more consistent results.

In [1]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

# Define the examples
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
]

# Define the prompt for a single example
example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template="Word: {word}\nAntonym: {antonym}"
)

# Create the FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Provide the antonym for the given word:",
    suffix="Word: {input}\nAntonym:",
    input_variables=["input"],
    example_separator="\n\n"
)

# Format the prompt with a new input
formatted_prompt = few_shot_prompt.format(input="big")
print(formatted_prompt)

Provide the antonym for the given word:

Word: happy
Antonym: sad

Word: tall
Antonym: short

Word: big
Antonym:


In [ ]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate
examples = [
    {"input": "What are the symptoms of flu?", "output": "Common symptoms include fever, cough, and body aches."},
    {"input": "How can I prevent the flu?", "output": "Get vaccinated and practice good hygiene."},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages(
    [   ("system", "You are a helpful assistant."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

llm = ChatGroq(model="qwen/qwen3-32b")
llm.invoke(final_prompt.format(input="What are the symptoms of flu?")).content

"<think>\nOkay, the user is asking again about the symptoms of the flu. Let me see what they previously asked. Earlier, they asked the same question and I responded that common symptoms include fever, cough, and body aches. Now they're asking the same thing again.\n\nHmm, maybe they want more details this time or perhaps they didn't get all the information they needed. I should check if there's a nuance in their question. The user might be looking for a more comprehensive list or want to know about less common symptoms. Let me recall the flu symptoms thoroughly.\n\nThe flu typically presents with fever, which is a high one, often above 100.4°F. Then there's the cough, which can be dry or productive. Body aches are indeed common, sometimes severe. They might also experience chills, which weren't mentioned before. Other symptoms include sore throat, runny or stuffy nose, fatigue, and sometimes headaches. Gastrointestinal symptoms like nausea, vomiting, or diarrhea can occur, especially i

: 